In [1]:
import json
import logging
import sys
import numpy as np
import pandas as pd
import greengrasssdk
# Create random name
import random
import string

In [2]:
MAX_NUM_CARPORT = 100
PUBLIC_CARPORT = 80
# init as free
carport_array = np.zeros(MAX_NUM_CARPORT, dtype=np.int8)
next_free_array = [i+1 for i in range(PUBLIC_CARPORT,MAX_NUM_CARPORT)]
temporary_veh_dic = {}
next_public_free = PUBLIC_CARPORT
num_occupied = 0

dataset = pd.DataFrame(
    columns=["vehicle_id", "user_id", "user_name", "veh_type", "veh_state", "registered_carport_id"])
# "veh_type": 1 registered, 2 temporary(guest)
# "veh_state": >=0: occupied carport id,
user_set = {}

In [3]:
def generate_user(id):
    global user_set
    if id not in user_set.keys():
        user_set['user_'+str(id)] = ''.join(
            [random.choice(string.ascii_letters + string.digits) for n in range(4)])


def generate_vehicle(id):
    global user_set
    veh_dic = {}
    veh_dic["vehicle_id"] = ''.join(
        [random.choice(string.ascii_letters + string.digits) for n in range(5)])
    if id not in user_set.keys():
        generate_user(id)
    user_id = 'user_'+str(id)
    veh_dic["user_id"] = user_id
    veh_dic["user_name"] = user_set[user_id]
    veh_dic["veh_type"] = 1
    veh_dic["veh_state"] = -1
    veh_dic["registered_carport_id"] = random.choice(range(PUBLIC_CARPORT))
    return veh_dic

def init_guest(veh_id):
    veh_dic = {}
    veh_dic["vehicle_id"] = veh_id
    veh_dic["user_id"] = -1
    veh_dic["user_name"] = "none"
    veh_dic["veh_type"] = 2
    veh_dic["veh_state"] = -1
    veh_dic["registered_carport_id"] = -1
    return veh_dic

def debug():
    global num_occupied, carport_array, next_free_array, next_public_free, user_set
    print("num_occupied", num_occupied)
    print("carport_array", carport_array)
    print("next_free_array", next_free_array)
    print("next_public_free", next_public_free)
    print("user_set", user_set)

In [4]:
def assign_carport(veh_type, veh_id):
    global num_occupied, carport_array, next_free_array, next_public_free, dataset
    assign_id = 0
    if num_occupied >= MAX_NUM_CARPORT:
        return -1
    if veh_type == 1:  # has registered carport
        filtered_df=dataset[dataset.vehicle_id == veh_id]
        # print(filtered_df)
        veh_dic={}
        veh_dic=filtered_df.to_dict(orient="list")
#         print(veh_dic)
        assign_id_list = veh_dic["registered_carport_id"]
        for assign_id in assign_id_list:
            if carport_array[assign_id] == 0:
                num_occupied += 1
                carport_array[assign_id]=1
                dataset.loc[dataset.vehicle_id == veh_id,"veh_state"] = assign_id
                return assign_id
    # not get the registered carport or guest car
    assign_id = next_public_free
    carport_array[next_public_free] = veh_type  # assign to a guest car
    next_public_free = next_free_array[assign_id-PUBLIC_CARPORT]
    num_occupied += 1
    dataset.loc[dataset.vehicle_id == veh_id,"veh_state"] = assign_id
    return assign_id


def unassign_carport(carport_id):
    global num_occupied, carport_array, next_free_array, next_public_free, dataset
    if carport_array[carport_id] == 0:
        return -1
    if carport_id >= PUBLIC_CARPORT:
        next_free_array[carport_id-PUBLIC_CARPORT] = next_public_free
        next_public_free = carport_id
    carport_array[carport_id] =0
    dataset.loc[dataset.veh_state == carport_id,"veh_state"]=-1
    num_occupied -= 1
    return 0

In [5]:
# init data(10 vehicles)
dataset.drop(dataset.index, inplace=True)
for i in range(10):
    dic = generate_vehicle(i)
#     print(dic)
    dataset = dataset.append(dic, ignore_index=True)
    assign_carport(1, dic["vehicle_id"])
#     print(dataset.head())

In [13]:
dataset.head(10)

,vehicle_id,user_id,user_name,veh_type,veh_state,registered_carport_id
0,o5oii,user_0,SEQK,1,47,47
1,0QTJf,user_1,H6yt,1,20,20
2,OEXKO,user_2,wYr3,1,65,65
3,qVCkU,user_3,u7yK,1,32,32
4,uM17Z,user_4,aBds,1,36,36
5,wEKtX,user_5,7ubW,1,23,23
6,Nq0i8,user_6,r1Da,1,49,49
7,Az1ok,user_7,o2y9,1,2,2
8,qdxka,user_8,oWxB,1,80,47
9,Mrpdz,user_9,yCss,1,25,25


In [14]:
def lambda_handler(event):
    global carport_array, dataset
    # controller
    if event["device"] == "controller":
        message = {"device": "cloud",
                   "behavior": "re_enter", "detail": "none"}
        veh_id = event["detail"]
        filtered_df = dataset[dataset.vehicle_id == veh_id]
        if filtered_df.empty:
            dic = init_guest(veh_id)
#             print(dic)
            dataset = dataset.append(dic, ignore_index=True)
            filtered_df = dataset[dataset.vehicle_id == veh_id]

        veh_dic = {}
        veh_dic = filtered_df.to_dict(orient="dict")
        print(veh_dic)
        if list(veh_dic["veh_state"].values())[0] >= 0:
            # quit
            message["behavior"] = "re_quit"
            ret = unassign_carport(list(veh_dic["veh_state"].values())[0])
            if ret < 0:
                # error
                message["detail"] = "error"
            else:
                message["detail"] = str(unassign_carport(list(veh_dic["veh_state"].values())[0]))
                if list(veh_dic["veh_type"].values())[0] == 2:
                    dataset.drop(index=list(veh_dic["veh_state"].keys())[0], inplace=True)
                    return
        else:
            # enter
            assign_id = assign_carport(list(veh_dic["veh_type"].values())[0], veh_id)
            if assign_id < 0:
                # error
                message["detail"] = "error"
            else:
                message["detail"] = str(assign_id)
    else:
        if event["behavior"] == "query":
            print("query")
            message = {"device": "cloud",
                       "behavior": "re_query", "detail": "none"}
            free_list = []
            occupied_list = []
            df1 = dataset[dataset.user_id == event["device"]]
            carport_id_list = df1["registered_carport_id"].to_list()
            for port in carport_id_list:
                if carport_array[port] == 0:
                    free_list.append(port)
                else:
                    occupied_list.append(port)
            for port in range(PUBLIC_CARPORT, MAX_NUM_CARPORT):
                if carport_array[port] == 0:
                    free_list.append(port)
                else:
                    occupied_list.append(port)
            message["detail"] = {"free_list": free_list,
                                 "occupied_list": occupied_list}
            print(message)

In [15]:
msg = {"device": "controller", "behavior": "enter", "detail": "aefr"}
lambda_handler (msg)
debug()
dataset.head(20)

{'vehicle_id': {10: 'aefr'}, 'user_id': {10: -1}, 'user_name': {10: 'none'}, 'veh_type': {10: 2}, 'veh_state': {10: -1}, 'registered_carport_id': {10: -1}}
num_occupied 11
carport_array [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
next_free_array [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
next_public_free 82
user_set {'user_0': 'SEQK', 'user_1': 'H6yt', 'user_2': 'wYr3', 'user_3': 'u7yK', 'user_4': 'aBds', 'user_5': '7ubW', 'user_6': 'r1Da', 'user_7': 'o2y9', 'user_8': 'oWxB', 'user_9': 'yCss'}


,vehicle_id,user_id,user_name,veh_type,veh_state,registered_carport_id
0,o5oii,user_0,SEQK,1,47,47
1,0QTJf,user_1,H6yt,1,20,20
2,OEXKO,user_2,wYr3,1,65,65
3,qVCkU,user_3,u7yK,1,32,32
4,uM17Z,user_4,aBds,1,36,36
5,wEKtX,user_5,7ubW,1,23,23
6,Nq0i8,user_6,r1Da,1,49,49
7,Az1ok,user_7,o2y9,1,2,2
8,qdxka,user_8,oWxB,1,80,47
9,Mrpdz,user_9,yCss,1,25,25


In [16]:
lambda_handler (msg)
debug()
dataset.head(20)

{'vehicle_id': {10: 'aefr'}, 'user_id': {10: -1}, 'user_name': {10: 'none'}, 'veh_type': {10: 2}, 'veh_state': {10: 81}, 'registered_carport_id': {10: -1}}
num_occupied 10
carport_array [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
next_free_array [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
next_public_free 81
user_set {'user_0': 'SEQK', 'user_1': 'H6yt', 'user_2': 'wYr3', 'user_3': 'u7yK', 'user_4': 'aBds', 'user_5': '7ubW', 'user_6': 'r1Da', 'user_7': 'o2y9', 'user_8': 'oWxB', 'user_9': 'yCss'}


,vehicle_id,user_id,user_name,veh_type,veh_state,registered_carport_id
0,o5oii,user_0,SEQK,1,47,47
1,0QTJf,user_1,H6yt,1,20,20
2,OEXKO,user_2,wYr3,1,65,65
3,qVCkU,user_3,u7yK,1,32,32
4,uM17Z,user_4,aBds,1,36,36
5,wEKtX,user_5,7ubW,1,23,23
6,Nq0i8,user_6,r1Da,1,49,49
7,Az1ok,user_7,o2y9,1,2,2
8,qdxka,user_8,oWxB,1,80,47
9,Mrpdz,user_9,yCss,1,25,25


In [17]:
msg = {"device": "user_1", "behavior": "query", "detail": "none"}
lambda_handler (msg)
debug()
dataset.head(20)

query
{'device': 'cloud', 'behavior': 're_query', 'detail': {'free_list': [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'occupied_list': [20, 80]}}
num_occupied 10
carport_array [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
next_free_array [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
next_public_free 81
user_set {'user_0': 'SEQK', 'user_1': 'H6yt', 'user_2': 'wYr3', 'user_3': 'u7yK', 'user_4': 'aBds', 'user_5': '7ubW', 'user_6': 'r1Da', 'user_7': 'o2y9', 'user_8': 'oWxB', 'user_9': 'yCss'}


,vehicle_id,user_id,user_name,veh_type,veh_state,registered_carport_id
0,o5oii,user_0,SEQK,1,47,47
1,0QTJf,user_1,H6yt,1,20,20
2,OEXKO,user_2,wYr3,1,65,65
3,qVCkU,user_3,u7yK,1,32,32
4,uM17Z,user_4,aBds,1,36,36
5,wEKtX,user_5,7ubW,1,23,23
6,Nq0i8,user_6,r1Da,1,49,49
7,Az1ok,user_7,o2y9,1,2,2
8,qdxka,user_8,oWxB,1,80,47
9,Mrpdz,user_9,yCss,1,25,25
